In [ ]:
import pandas as pd # Pour manipuler des dataframes
import numpy as np # Pour manipuler des arrays
import matplotlib.pyplot as plt # Pour afficher des graphiques
import seaborn as sns # Pour améliorer l'affichage des graphiques
import duckdb # Pour manipuler des bases de données
from datetime import datetime # Pour manipuler des dates

## LECTURE + INSERTION BDD

In [ ]:
# Connexion ou création de la base de données
con = duckdb.connect("my_database.duckdb")

# Liste des fichiers Parquet à insérer
files = [
    'data/2019-Oct.parquet',
    'data/2019-Nov.parquet',
    'data/2019-Dec.parquet',
    'data/2020-Jan.parquet',
    'data/2020-Feb.parquet',
    'data/2020-Mar.parquet',
    'data/2020-Apr.parquet'
]

# Créer la table si elle n'existe pas
con.execute("""
    CREATE TABLE IF NOT EXISTS data AS SELECT * FROM parquet_scan('{}') WHERE 1=0
""".format(files[0]))

# Insérer les fichiers un par un
for file in files:
    print(f"Inserting data from {file}, starting time : {datetime.now()}")
    con.execute(f"""
        INSERT INTO data SELECT * FROM parquet_scan('{file}');
    """)

print("Data successfully inserted!")

# Vérification (optionnel)
sample_data = con.execute("SELECT * FROM data LIMIT 5").fetchall()
print("Sample Data:", sample_data)

# Fermer la connexion
con.close()

## NETTOYAGE

Nettoyage selon plusieurs règles:
- Suppression de toutes les lignes où brand = 'None' ou NULL, (50 000 000 lignes)

In [ ]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Comptage des lignes initiales dans la table 'data'
initial_count = con.execute("SELECT COUNT(*) FROM data").fetchall()[0][0]
print(f"Initial number of rows: {initial_count}")

# Créer une table temporaire sans les lignes à supprimer (celles qui ont 'brand' NULL ou 'None')
con.execute("""
    DELETE FROM data
    WHERE (brand IS NULL OR brand = 'None');
""")

# Comptage des lignes restantes dans la table temporaire
final_count = con.execute("SELECT COUNT(*) FROM data").fetchall()[0][0]

# Calcul du nombre de lignes supprimées
deleted_count = initial_count - final_count

# Afficher le nombre de lignes supprimées
print(f"Number of rows deleted: {deleted_count}")

# Fermer la connexion
con.close()


## Création de la table User

In [ ]:
# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")


# Créer une table 'Utilisateur' avec les informations agrégées
con.execute("""
    CREATE TABLE Utilisateur AS
    SELECT
        user_id AS identifiant_utilisateur,
        -- Calcul du total dépensé par l'utilisateur
        SUM(CASE WHEN price IS NOT NULL THEN price ELSE 0 END) AS total_depense,
        
        -- Calcul du nombre total d'achats effectués
        COUNT(DISTINCT CASE WHEN price > 0 THEN user_session END) AS nombre_total_achats,
        
        -- Calcul du nombre total de vues effectuées (par session)
        COUNT(DISTINCT CASE WHEN price IS NULL THEN user_session END) AS nombre_total_vues,
        
        -- Calcul du nombre de catégories uniques vues
        COUNT(DISTINCT category_code) AS categories_uniques_vues,
        
        -- Calcul du nombre de produits uniques vus
        COUNT(DISTINCT product_id) AS produits_uniques_vus,
        
        -- Calcul du nombre total de sessions de l'utilisateur
        COUNT(DISTINCT user_session) AS total_sessions,
        
        -- Calcul de l'achat moyen
        AVG(CASE WHEN price > 0 THEN price ELSE NULL END) AS achat_moyen,
        
        -- Vérifier si l'utilisateur a acheté un ebook (dans category_code)
        MAX(CASE WHEN category_code = 'computer.ebooks' THEN 1 ELSE 0 END) AS a_achete_des_ebooks
        
    FROM
        data
    GROUP BY
        user_id;
""")

# Affichage du nombre de lignes insérées dans la nouvelle table 'Utilisateur'
nombre_lignes_nouvelle_table = con.execute("SELECT COUNT(*) FROM Utilisateur").fetchall()[0][0]
print(f"Nombre de lignes insérées dans la table Utilisateur : {nombre_lignes_nouvelle_table}")

# Fermer la connexion
con.close()
